# `DRAP`

### Database of Remote Affective Physiological Signals and Continuous Ratings Collected in Virtual Reality

Notebook containing the postprocessing stages for the DRAP dataset.

It transforms individual files containing events (.json) and physiological responses (.csv) per each participant, and produces a single file `Dataset_DRAP_full_postprocessed.csv` synchronizing: 

1) Amplitude from Physiological responses. 
2) Affect ratings.
3) Start and End of intervention stages Resting and Video for three types of video content: 
    - VideoNegative
    - VideoPositive
    - VideoNeutral

In [1]:
# Add files to sys.path
from pathlib import Path
import sys,os
this_path = None
try:    # WORKS WITH .py
    this_path = str(os.path.dirname(os.path.abspath(__file__)))
except: # WORKS WITH .ipynb
    this_path = str(Path().absolute())+"/" 
print("File Path:", this_path)

# Add the level up to the file path so it recognizes the scripts inside `drap`
sys.path.append(os.path.join(this_path, ".."))

File Path: e:\dsv\dev\git_repos\DRAP\notebooks/


In [2]:
# Import classes
from drap.utils import files_handler  # Utils for generation of files and paths
import drap.preprocessing       # Generate dataset index, load files, and plots.

# Import data science libs
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
#matplotlib.rcParams['text.usetex'] = True

---
## SETUP

In [3]:
### General configuration

# Path to the participants' folder w.r.t this notebook's filepath
DATASET_ROOT_FOLDER = "../data/"

# Used to generate the path of temporary subfolders
DATASET_NAME = "DRAP"

In [4]:
# Functions to generate filepaths

# MAIN FOLDERS FOR OUTPUT FILES
ROOT = this_path + ""   # Root folder for all the files w.r.t this file
TEMP_FOLDER = ROOT+"temp/"  # Main folder for temp files with intermediate calculations
RESULTS_FOLDER = ROOT+"results/"    # Folder to recreate plots and results from analyses

# Generates paths for the temporary files created from this script
def gen_path_temp(filename, extension, subfolders=""):
    # Generates full paths for TEMP FILES just by specifying a name
    return files_handler.generate_complete_path(filename, \
                                        main_folder=TEMP_FOLDER, \
                                        subfolders=DATASET_NAME+"/"+subfolders, \
                                        file_extension=extension)

---
# Load and preprocess datasets

In [5]:
data_loader = drap.preprocessing.Manager(DATASET_ROOT_FOLDER, 
                                    index_files_path = TEMP_FOLDER, #TEMP_FOLDER, # None,
                                    force_index_regeneration=True, 
                                    verbose = True,
                                    )

Forcing index construction:  e:\dsv\dev\git_repos\DRAP\notebooks/temp/drap_index/drap_tree_index.json

Directory >> participant_101
	 Data>> fast_movement
	 Event>> fast_movement
	 Data>> slow_movement
	 Event>> slow_movement
	 Data>> video_1
	 Event>> video_1
	 Data>> video_2
	 Event>> video_2
	 Data>> video_3
	 Event>> video_3
	 Data>> video_4
	 Event>> video_4
	 Data>> video_5
	 Event>> video_5
	 Events compiled in e:\dsv\dev\git_repos\DRAP\notebooks/temp/drap_index/participant_101\compiled_emotion_ratings.csv

Directory >> participant_216
	 Data>> fast_movement
	 Event>> fast_movement
	 Data>> slow_movement
	 Event>> slow_movement
	 Data>> video_1
	 Event>> video_1
	 Data>> video_2
	 Event>> video_2
	 Data>> video_3
	 Event>> video_3
	 Data>> video_4
	 Event>> video_4
	 Data>> video_5
	 Event>> video_5
	 Events compiled in e:\dsv\dev\git_repos\DRAP\notebooks/temp/drap_index/participant_216\compiled_emotion_ratings.csv

Directory >> participant_219
	 Data>> fast_movement
	 Event>> f

In [ ]:
data_loader.index[0]

In [ ]:
data_loader.events[0]

In [ ]:
data_loader.segments[0]

In [ ]:
df_sum = None
for pid,pdata in data_loader.index.items():
    print(f"Participant {pid} with folder id: {pdata['participant_id']} was part of protocol: {pdata['protocol']}")

    # Summary grouped per session segment    
    for segtype in data_loader.events[pid]["Session"].unique():
        
        ######
        # Summary of events
        events_data = data_loader.events[pid]
        Q = ( (events_data["Session"] == segtype) )
        event_filtered = events_data[Q].reset_index()

        ######
        # Summary of emotions
        emotions_data = data_loader.emotions[pid]
        Q = ( (emotions_data["Session"] == segtype) )
        emotions_filtered = emotions_data[Q].reset_index()

        # Summarize dataframe
        current_summary = {
                    # "index_id": pid,
                    # "participant_id": pdata['participant_id'],
                    # "protocol": pdata['protocol'],
                    "Segment":segtype,
                    "Events_N": event_filtered.shape[0],
                    "Events_duration": event_filtered["Time"].iloc[-1] - event_filtered["Time"].iloc[0],
                    "Emotions_N": emotions_filtered.shape[0],
                    "Emotions_duration": (np.array(emotions_filtered["Time"])[-1] - np.array(emotions_filtered["Time"])[0]) if emotions_filtered.shape[0]>0 else np.nan,
                    "Emotions_Valence_avg": emotions_filtered["Valence"].mean(),
                    "Emotions_Arousal_avg": emotions_filtered["Arousal"].mean()
                }

        # Convert from values to list, to adapt to DataFrame
        current_summary = { k:[v] for k,v in current_summary.items() }
        current_summary = pd.DataFrame.from_dict(current_summary)

        df_sum = current_summary if (df_sum is None) else pd.concat([df_sum, current_summary], ignore_index=True)
    
    # Insert participant's data
    current_summary.insert(0, "protocol", value=pdata['protocol'])
    current_summary.insert(0, "participant_id", value= pdata['participant_id'])
    current_summary.insert(0, "index_id", value=pid)
    
# Print
df_sum

In [ ]:
# Load experimental events data
# data_loader.load_event_files()
for id,events in data_loader.events.items():
    print(f"Participant {id} with folder id: {data_loader.index[id]['participant_id']} has in total # events: {events.shape[0]}")

In [ ]:
data_loader.emotions[0]

In [ ]:
# Load emotions data
data_loader.load_emotion_files()
for id,emotions in data_loader.emotions.items():
    print(f"Participant {id} with folder id: {data_loader.index[id]['participant_id']} has in total # self-reported emotional values: {emotions.shape[0]}")

In [ ]:
data_loader.events[0].head()

In [ ]:
data_loader.emotions[0].head()

In [ ]:
# Load segments data
data_loader.load_segments_files()
for id,segments in data_loader.segments.items():
    print(f"Participant {id} with folder id: {data_loader.index[id]['participant_id']} has in total # segments: {segments.shape[0]}")

In [ ]:
data_loader.segments[0].head()

Sensors placement

![EmteqMaskSensors](./datasets/EmteqlabReferenceSensors.jpg)

In [ ]:
# Example to load all variables types from a single facial EMG muscle Center Corrugator
COLNAMES_CENTER_CORRUGATOR=DRAP.GetColnamesFromEmgMuscle(DRAP.EmgMuscles.CenterCorrugator)
data, metadata = data_loader.load_data_from_participant(participant_idx = 0, session_part="video_1", columns = COLNAMES_CENTER_CORRUGATOR)
data.head()

In [ ]:
# Get the column names for all variables of interest in the rest of the analysis
VARS_OF_INTEREST = DRAP.GetColnamesBasicsNonEmg() + DRAP.GetColnamesFromEmgVariableType(DRAP.EmgVars.Amplitude)
VARS_OF_INTEREST

In [ ]:
data, metadata = data_loader.load_data_from_participant(participant_idx = 0, session_part="video_1", columns = VARS_OF_INTEREST)
data.head()

In [ ]:
metadata

In [ ]:
data.describe()

In [ ]:
# Exploratory visualization of all data samples
data.iloc[2000:].plot.line(subplots=True, figsize=(20,1*data.shape[1]), sharex=True)

## Iterating over all participants

In [ ]:
# Total participants
participants_ids = data_loader.index.keys()
participants_ids

In [ ]:
# Total sessions
experimental_stages_ids = [ str(session) for session in data_loader.SessionSegment ]
experimental_stages_ids

In [ ]:
# ### Test to know how long it would take to load all the dataset of interest
# import time
# for participant in participants_ids:
#     for exp_stage in experimental_stages_ids:
#         t0 = time.time()
#         data, metadata = data_loader.load_data_from_participant(participant_idx = participant, session_part = exp_stage, columns=VARS_OF_INTEREST)
#         print(f"\t>> Loading time: {time.time()-t0} s")

# print("\n\n=======\nFinished loading all relevant data!") 

# ### It took around 4.5mins just loading all the datasets

In [ ]:
# Extract the sequence order of the videos

for SYNC_TIME_USER_ID in participants_ids:
    MASK_QUERY = ( 
                # data_loader.events[SYNC_TIME_USER_ID].Event.str.startswith( "Playing category number: " ) | \
                    # data_loader.events[SYNC_TIME_USER_ID].Event.str.startswith( "Video category finished" ) | \
                        # data_loader.events[SYNC_TIME_USER_ID].Event.str.startswith( "Playing rest video" ) | \
                            # data_loader.events[SYNC_TIME_USER_ID].Event.str.startswith( "Finished playing rest video" ) \
                    data_loader.events[SYNC_TIME_USER_ID].Event.str.startswith( "Playing" ) \
                    )
    
    # Event sequence
    EVENT_TEXT_SEQUENCE = "Category sequence:"
    keys_containing_sync_event = data_loader.events[SYNC_TIME_USER_ID].Event.str.startswith(EVENT_TEXT_SEQUENCE)
    cat_sequence = data_loader.events[SYNC_TIME_USER_ID][ keys_containing_sync_event ].iloc[0] # Choose first event
    video_seq = cat_sequence.Event.split(":")[1].split(",")

    print(f"Participant: {SYNC_TIME_USER_ID}, Events: {data_loader.events[SYNC_TIME_USER_ID][MASK_QUERY].shape}, Seq: {video_seq}")


## Testing pipeline to merge physiological and continuous affective ratings

The merging pipeline assumes a Participant ID, and the experimental stage to process (*VideoNegative, VideoNeutral, VideoPositive*)
1. Identify the timestamps for the resting stage $[r_{t0},r_{t1}]$ and the stage watching the video $[v_{t0},v_{t1}]$
2. Find the VideoID of the content being watched at each moment (facilitates filtering per video, if desired)
3. Merge the physiological and emotional data with corresponding timestamps.
4. Resample the dataframes at 50Hz
5. Save the merged dataset in a CSV file

In [ ]:
## Participant and Video stage to process
PARTICIPANT_IDX = 7
EXPERIMENTAL_STAGE_NAME = "VideoPositive"

Identify timestamps dividing resting and video stages within segment

In [ ]:
def calculate_info_from_segment(df_segments, segment_name_to_filter):
    """
    Processes a dataframe of segments timestamps and returns a tuple with:
        - rest_tstamp_start
        - rest_tstamp_end
        - video_tstamp_start
        - video_tstamp_end
        - video_filename
    """

    # Filter the segment corresponding to the intended video
    df_segments = df_segments[ df_segments.Segment == segment_name_to_filter]

    # Find the beginning and end of the RESTING (VideoId == -1)
    rest_start = df_segments[ (df_segments.Trigger=="Start") & (df_segments.VideoId == -1)].index.min()
    rest_end = df_segments[ (df_segments.Trigger=="End") & (df_segments.VideoId == -1)].index.max()

    # The segment watching the VIDEO (VideoId != -1)
    video_start = df_segments[ (df_segments.Trigger=="Start") & (df_segments.VideoId != -1)].index.min()
    video_end = df_segments[ (df_segments.Trigger=="End") & (df_segments.VideoId != -1)].index.max()

    # Which file should be loaded to access the required data
    video_filename = df_segments.Session.iloc[0]

    # Correct the few situations when the video starts before resting ends for few miliseconds
    if rest_end > video_start:
        video_start = rest_end
    
    return (rest_start, rest_end, video_start, video_end, video_filename)

In [ ]:
r_t0, r_t1, v_t0, v_t1, video_filename = calculate_info_from_segment(data_loader.segments[PARTICIPANT_IDX], EXPERIMENTAL_STAGE_NAME)
print(f"Participant: \t\t{PARTICIPANT_IDX} \nRest duration: \t\t{r_t1-r_t0}s, \nVideos duration: \t{v_t1-v_t0} \nVideoName: \t\t{video_filename} \nResting was first: \t{r_t0 < v_t0}")

In [ ]:
# Load video corresponding to desired Experimental stage
data, metadata = data_loader.load_data_from_participant(participant_idx = PARTICIPANT_IDX, session_part = video_filename, columns = VARS_OF_INTEREST)

Filter experimental stages

In [ ]:
# Filter experimental stages
data_rest = data[ (data.index >= r_t0) & (data.index < r_t1) ]
data_video = data[ (data.index >= v_t0) & (data.index < v_t1) ]

print(data_rest.shape, data_video.shape)
print("Duration stage REST: ", r_t1-r_t0)
print("Duration stage VIDEO: ", v_t1-v_t0)

In [ ]:
# data_video.plot.line(subplots=True, figsize=(15,1*data.shape[1]), sharex=True)

Load emotional responses within video range

In [ ]:
# Emotions reported during the video, and in the corresponding video
Q = (data_loader.emotions[PARTICIPANT_IDX].index >= v_t0) & \
        (data_loader.emotions[PARTICIPANT_IDX].index < v_t1 ) & \
        (data_loader.emotions[PARTICIPANT_IDX].Session == video_filename)
        
data_emotions = data_loader.emotions[PARTICIPANT_IDX][ Q ].drop("Session", axis=1)
data_emotions.head()

Find the VideoId per timestamp

In [ ]:
def calculate_video_id_end_timestamps(df_segments, segment_name_to_filter):
    """
    Returns a dataframe with the timestamp where a given VideoID *finishes*.
        - df_segments = DataFrame with segments
    """
    # EVENT_TEXT_SEQUENCE = "Finished playing video number:" # It will return when the event finished!
    # keys_containing_sync_event = df_events.Event.str.startswith(EVENT_TEXT_SEQUENCE)
    # videos_seq = df_events[ keys_containing_sync_event ] # Choose all video numbers
    # videos_ending = videos_seq.Event.str.split(":")
    # video_id_end_timestamp = videos_ending.apply((lambda x: int(x[1])))
    # video_id_end_timestamp = pd.DataFrame({"VideoID":video_id_end_timestamp})

    # Filter the segment corresponding to the intended video
    df_segments = df_segments[ df_segments.Segment == segment_name_to_filter]

    # Find the end of each video stage
    video_id_end_timestamp = df_segments[ (df_segments.Trigger=="End") ]
    video_id_end_timestamp = video_id_end_timestamp[ ["VideoId"] ]
    
    return video_id_end_timestamp

In [ ]:
video_id_end_timestamp = calculate_video_id_end_timestamps(data_loader.segments[PARTICIPANT_IDX], EXPERIMENTAL_STAGE_NAME)
video_id_end_timestamp

In [ ]:
video_id_end_timestamp.index

In [ ]:
print(r_t0, r_t1)
print(v_t0, v_t1)

In [ ]:
data_rest = pd.merge_asof(data_rest, video_id_end_timestamp, left_index=True, right_index=True, direction="forward")
data_rest["VideoId"].value_counts()

In [ ]:
data_video = pd.merge_asof(data_video, video_id_end_timestamp, left_index=True, right_index=True, direction="forward")
data_video["VideoId"].value_counts()

In [ ]:
data_rest.insert(0, "OriginalParticipantID", data_loader.index[PARTICIPANT_IDX]['folderid'])
data_video.insert(0, "OriginalParticipantID", data_loader.index[PARTICIPANT_IDX]['folderid'])
data_video

Merging physiology with affective ratings

In [ ]:
# Merge physio with subjective emotions
data_merged = pd.merge_asof(data_video, data_emotions, left_index=True, right_index=True)

In [ ]:
print("RANGE VIDEO: \t\t", data_video.index.min(), "---", data_video.index.max(), " \tLength:", data_video.index.max()-data_video.index.min())
print("RANGE EMOTIONS: \t", data_emotions.index.min(), "---", data_emotions.index.max(), " \tLength:", data_emotions.index.max()-data_emotions.index.min())

In [ ]:
# data_merged.plot.line(subplots=True, figsize=(15,1*data.shape[1]), sharex=True)

Resample data at 50Hz

In [ ]:
def calculate_resampled_dataframe(df, sampling_frequency_hz = 50):
    df.index = pd.to_datetime(df.index, unit="s")
    _FS = sampling_frequency_hz
    df_resampled = df.resample(str(1/_FS)+'S', origin='start').ffill()
    # The valence, arousal, rawX, rawY will contain null values before the first value is captured. Fill with first value.
    df_resampled = df_resampled.fillna(method="backfill")
    # Put the data back to 0 seconds
    df_resampled.index -= df_resampled.index[0]
    # Transform from datetime to float
    df_resampled.index = df_resampled.index.total_seconds()
    return df_resampled

In [ ]:
# Resample to 50Hz
FS = 50
data_total = calculate_resampled_dataframe(data_merged, FS)
data_total.head()

In [ ]:
# Missing values
data_total.isnull().sum()

In [ ]:
data_total.plot.line(subplots=True, figsize=(15,1*data.shape[1]), sharex=True)#[0].figure.savefig(gen_path_temp(f"fig_test_data", extension=".png"))

Testing the merging process with fake data

In [ ]:
# Merge two fake dataframes with MultiIndex
data2 = data_total.copy(deep=True)
data3 = data_total.copy(deep=True)

# Adding index with participant and video category
data_total = pd.concat({(PARTICIPANT_IDX,EXPERIMENTAL_STAGE_NAME):data_total}, names = ["Participant","VideoCategory"])
data_total.head()
# Toy data to prove how to merge fake "VideoPositive" data vertically
data2 = pd.concat({(PARTICIPANT_IDX,"VideoPositive"):data2}, names = ["Participant","VideoCategory"])
data_total = pd.concat([data_total, data2])
# Toy data to combine fake "VideoNeutral" vertically
data3 = pd.concat({(PARTICIPANT_IDX,"VideoNeutral"):data3}, names = ["Participant","VideoCategory"])
data_total = pd.concat([data_total, data3])
data_total

In [ ]:
# Save data from a single participant and single experimental session
data_total.to_csv(gen_path_temp("example_df",extension=".csv"))

## Create dataset merging all data

Loading all segments for all participants and store the resting and video parts in a single large CSV

In [ ]:
# Where the compiled dataset will be stored
DATASET_POSTPROCESSED_FILENAME = gen_path_temp("Dataset_DRAP_full_postprocessed", extension=".csv")

# Besides the compiled dataset in .CSV, it generates a folder with individual datasets per 
# participant, and with plots that show the saved data
SAVE_SINGLE_FILES_AND_PLOTS = False

# Segments of interest. The timestamps that determine each stage will be found and used to segment the physiological data per participant.
VIDEO_SEGMENT_NAMES = ["VideoNegative", "VideoPositive", "VideoNeutral"]    # Key of experimental stages
PREFIX_RESTING_STAGE = "Resting_"
FS = 50 # Sampling frequency

# Load or create dataframe with statistics of initial dataset
dataset_postprocessed_final = None

### INPUTS / OUTPUTS
"""EDIT CUSTOM FILENAMES"""
input_files = [DATASET_POSTPROCESSED_FILENAME]

# Try to load files maximum two times
RELOAD_TRIES = 2
for tries in range(RELOAD_TRIES):
    try:
        ### LOAD FILE
        print(f"Trying {tries+1}/{RELOAD_TRIES} to load files: {input_files}")
        
        ### CUSTOM SECTION TO READ FILES
        """EDIT CUSTOM READ"""
        dataset_postprocessed_final = pd.read_csv(input_files[0])#, index_col=[0,1,2])
        print(f"File {input_files[0]} was successfully loaded")

    except Exception as e:
        ### CREATE FILE
        print(f"File not found. Creating again! {e}")

        ### CUSTOM SECTION TO CREATE FILES 
        """EDIT CUSTOM WRITE"""

        for PARTICIPANT_IDX in participants_ids:
            for EXPERIMENTAL_STAGE_NAME in VIDEO_SEGMENT_NAMES:

                ## Extract segments for specific video type
                r_t0, r_t1, v_t0, v_t1, video_filename = calculate_info_from_segment(data_loader.segments[PARTICIPANT_IDX], EXPERIMENTAL_STAGE_NAME)
                print(f"\n\nParticipant: \t\t{PARTICIPANT_IDX} \nRest range: \t\t{r_t1-r_t0}s, \nVideos range: \t{v_t1-v_t0} \nVideo filename: \t\t{video_filename} \nResting was first: \t{r_t0 < v_t0}")

                # Load corresponding data and metadata
                data, metadata = data_loader.load_data_from_participant(participant_idx = PARTICIPANT_IDX, session_part = video_filename, columns = VARS_OF_INTEREST)
                
                # Detect the ending timestamp of each VideoID to be added to the datasets
                video_id_end_timestamp = calculate_video_id_end_timestamps(data_loader.segments[PARTICIPANT_IDX], EXPERIMENTAL_STAGE_NAME)
                
                """ PROCESSING DATA FROM RESTING STAGES """
                # Filter experimental stages
                data_rest = data[ (data.index >= r_t0) & (data.index < r_t1) ]
                # Combine the videoId with the data from the segment
                data_rest = pd.merge_asof(data_rest,  video_id_end_timestamp, left_index=True, right_index=True, direction="forward")

                # Emotions reported during the video, and in the corresponding video
                Q = (data_loader.emotions[PARTICIPANT_IDX].index >= r_t0) & \
                        (data_loader.emotions[PARTICIPANT_IDX].index < r_t1 ) & \
                        (data_loader.emotions[PARTICIPANT_IDX].Session == video_filename)
                data_emotions_rest = data_loader.emotions[PARTICIPANT_IDX][ Q ].drop("Session", axis=1)
                # Merge data end emotions in a single dataframe per time
                data_rest = pd.merge_asof(data_rest, data_emotions_rest, left_index=True, right_index=True)

                # Resampling data
                data_rest_resampled = calculate_resampled_dataframe(data_rest, FS)

                """ PROCESSING DATA FROM VIDEO STAGES """
                # Filter experimental stages
                data_video = data[ (data.index >= v_t0) & (data.index < v_t1) ]
                # Combine the videoId with the data from the segment
                data_video = pd.merge_asof(data_video, video_id_end_timestamp, left_index=True, right_index=True, direction="forward")
                
                # Emotions reported during the video, and in the corresponding video
                Q = (data_loader.emotions[PARTICIPANT_IDX].index >= v_t0) & \
                        (data_loader.emotions[PARTICIPANT_IDX].index < v_t1 ) & \
                        (data_loader.emotions[PARTICIPANT_IDX].Session == video_filename)
                data_emotions = data_loader.emotions[PARTICIPANT_IDX][ Q ].drop("Session", axis=1)
                
                # Merge data end emotions in a single dataframe per time
                data_video = pd.merge_asof(data_video, data_emotions, left_index=True, right_index=True)
                # Resample dataset to constant period
                data_video_resampled = calculate_resampled_dataframe(data_video, FS)

                """ COMBINING DATASET IN A SINGLE ONE """
                print(f"Actual duration stage VIDEO: {data_video_resampled.index.max()} \tSHORT?:{data_video_resampled.index.max()<295}")
                print(f"Actual duration stage REST: {data_rest_resampled.index.max()} \tSHORT?:{data_rest_resampled.index.max()<115}")
                print(f"Total missing vals:{data_video_resampled.isnull().sum().sum()}")

                # Add a column with the original participant ID corresponding to the original dataset
                folder_id = data_loader.index[PARTICIPANT_IDX]['folderid']
                data_rest_resampled.insert(0, "OriginalParticipantID", folder_id)
                data_video_resampled.insert(0, "OriginalParticipantID", folder_id)

                ### SAVING FILES
                if SAVE_SINGLE_FILES_AND_PLOTS:
                    # Save resting data
                    data_to_plot = {
                                        "video_data":data_video_resampled,
                                        "resting_data":data_rest_resampled
                                    }
                    for _df_name,_df in data_to_plot.items():
                        # Save video data
                        save_path_plot = gen_path_temp(f"per_participant/_plots/{folder_id}/{EXPERIMENTAL_STAGE_NAME}_{_df_name}", extension=".png")
                        save_path_csv = gen_path_temp(f"per_participant/{folder_id}/{EXPERIMENTAL_STAGE_NAME}_{_df_name}", extension=".csv")
                        _df.plot.line(subplots=True, figsize=(15,1*_df.shape[1]), sharex=True)[0].figure.savefig(save_path_plot); plt.close()
                        _df.to_csv(save_path_csv)

                ### Generating multiindex to create a single .csv with all the data
                COLNAMES_MULTIINDEX = ["Participant","Stage"]
                data_video_resampled = pd.concat({(PARTICIPANT_IDX,EXPERIMENTAL_STAGE_NAME):data_video_resampled}, names = COLNAMES_MULTIINDEX)
                data_rest_resampled = pd.concat({(PARTICIPANT_IDX, PREFIX_RESTING_STAGE + EXPERIMENTAL_STAGE_NAME):data_rest_resampled}, names = COLNAMES_MULTIINDEX)

                # Final concatenation of resting and video stages
                data_compiled = pd.concat([data_video_resampled.copy(deep=True), data_rest_resampled.copy(deep=True)])

                # Generate final DF
                if(dataset_postprocessed_final is None):
                    dataset_postprocessed_final = data_compiled.copy(deep=True)
                    # dataset_postprocessed_final = pd.concat([dataset_postprocessed_final, data_rest_resampled.copy(deep=True)])
                else:
                    dataset_postprocessed_final = pd.concat([dataset_postprocessed_final, data_compiled.copy(deep=True)])
                    # dataset_postprocessed_final = pd.concat([dataset_postprocessed_final, data_rest_resampled.copy(deep=True)])

        # Saving .csv
        dataset_postprocessed_final.to_csv( DATASET_POSTPROCESSED_FILENAME )
        print("\n\n End")


        ### ---- CONTROL RETRIES
        if tries+1 < RELOAD_TRIES:
            continue
        else:
            raise
    
    # Finish iteration
    break

In [ ]:
dataset_postprocessed_final.head()

In [ ]:
dataset_postprocessed_final.shape

In [ ]:
print(">> FINISHED WITHOUT ERRORS!!")